# Setup


In [1]:
from google.colab import drive
drive.mount('/content/drive')

folder = f"/content/drive/MyDrive/DH"

import os
import time
import json,math,sys,re
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, f1_score, cohen_kappa_score,recall_score
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import notebook as tqdm

KeyboardInterrupt: 

In [2]:
stylized_to_standard = {'ᴇ': 'E','ꜱ': 'S','ᴠ': 'V','ᴅ': 'D','ʀ': 'R','ᴍ': 'M','ɪ': 'I','ᴛ':
                        'T','ʏ': 'Y','ʟ': 'L','ᴘ': 'P','ᴀ': 'A','ɴ': 'N','ꞵ': 'B','ʙ': 'B',
                        'ꞽ': 'Y','ʜ': 'H','Æ': 'AE','ꜰ': 'F','æ': 'ae','ᴏ': 'O','ᴡ': 'W','ᴄ':
                        'C','ᴋ': 'K','ɢ': 'G','ᴢ': 'Z','é': 'e','ô': 'o','ë': 'e','ó': 'o',
                        'á': 'a','œ': 'oe','ȯ': 'o','³': '3','Ꜳ': 'AE','Ꜵ': 'AO','Ꜷ': 'AU'}

def normalize_stylized(text):
    return ''.join(stylized_to_standard.get(char, char) for char in text)


def fix_name(v_id):
  if "1 Kings" in v_id: v_id = "3 Kings" + v_id.split("1 Kings")[-1]
  elif "2 Kings" in v_id: v_id = "4 Kings" + v_id.split("2 Kings")[-1]
  elif "1 Samuel" in v_id: v_id = "1 Kings" + v_id.split("1 Samuel")[-1]
  elif "2 Samuel" in v_id: v_id = "2 Kings" + v_id.split("2 Samuel")[-1]
  elif re.search(r"^\d+ Chronicles",v_id): v_id = re.sub(r"Chronicles","Paralipomenon",v_id)
  return v_id

In [3]:
bible = {}
remove = []
# special = {}

b_versions = ['AKJV','ODRV','Geneva', 'Douay-Rheims', 'Tyndale', 'Wycliffe','Vulgate']
for bname in b_versions:
    data = pd.read_csv(f"{folder}/Early-Modern-Sermons/assets/Bibles/{bname}.csv",header=None)
    data = data.to_dict(orient="records")
    for entry in tqdm.tqdm(data):
        key = entry[0]
        v_id = key.split(" (")[0]
        text = entry[6]
        # text = re.sub(r'\s+', ' ', text).strip()
        # text = normalize_stylized(text)
        # special.update(find_unicode_chars(text))
        # entry[6] = text
        if re.search("Douay-Rheims",key):
            if re.sub("Douay-Rheims","ODRV",key) in bible: continue
        if len(text.split(" ")) < 200:
            bible[key] = f"{v_id} {text}"

        parts = re.split(r'(?<=[\.\?]) (?=[A-Z])|(?<=[\!\:\;])', text)
        parts = [re.sub(r'\s+', ' ', p).strip() for p in parts if len(p.strip(" ")) > 0]
        if (len(parts[0].split(" ")) <= 5 or len(parts[-1].split(" ")) <= 5 or re.search(r"\&\w+\;",parts[0])):
            for pidx, p in enumerate(parts): continue
        elif len(parts) > 1:
            for pidx, p in enumerate(parts):
              p_id = f"{key} - {pidx}"
              if len(p) == 0: continue
              if re.search(r"\&\w+\;",p) or len(p.split(" ")) <= 5: continue
              bible[p_id] = f"Part {pidx+1} of {v_id}: {p}"

bible['NonQP 0.0'] = "No Biblical quotation or paraphrase found"

bible_verses = list(bible.values())
bible_ids = list(bible.keys())
id_to_idx = {v_id:idx for idx, v_id in enumerate(bible_ids)}
len(bible_verses)

  0%|          | 0/36702 [00:00<?, ?it/s]

  0%|          | 0/14737 [00:00<?, ?it/s]

  0%|          | 0/31090 [00:00<?, ?it/s]

  0%|          | 0/35811 [00:00<?, ?it/s]

  0%|          | 0/7954 [00:00<?, ?it/s]

  0%|          | 0/9622 [00:00<?, ?it/s]

  0%|          | 0/35809 [00:00<?, ?it/s]

276628

In [4]:
for v_id in bible:
    if re.search("^Genesis 1.4 ",v_id):
        # if " - " in v_id: continue
        # if "AKJV" not in v_id and "Geneva" not in v_id: continue
        # if "ODRV" not in v_id and "Vulgate" not in v_id: continue
        # if " - 0" not in v_id: continue
        # print(f'{v_id}\t\t{bible_info[v_id][3]}')
        print(v_id, "\n",f'{bible[v_id]}','\n')

Genesis 1.4 (AKJV) 
 Genesis 1.4 And God saw the light, that it was good: and God diuided the light from the darkenesse. 

Genesis 1.4 (AKJV) - 0 
 Part 1 of Genesis 1.4: And God saw the light, that it was good: 

Genesis 1.4 (AKJV) - 1 
 Part 2 of Genesis 1.4: and God diuided the light from the darkenesse. 

Genesis 1.4 (ODRV) 
 Genesis 1.4 And God saw the light that it was good: & he diuided the light from the darkenes. 

Genesis 1.4 (ODRV) - 0 
 Part 1 of Genesis 1.4: And God saw the light that it was good: 

Genesis 1.4 (ODRV) - 1 
 Part 2 of Genesis 1.4: & he diuided the light from the darkenes. 

Genesis 1.4 (Geneva) 
 Genesis 1.4 And God saw the light that it was good, and God separated the light from the darkenes. 

Genesis 1.4 (Wycliffe) 
 Genesis 1.4 And God seiy the liyt, that it was good, and he departide the liyt fro derknessis; and he clepide the liyt, 

Genesis 1.4 (Vulgate) 
 Genesis 1.4 Et vidit Deus lucem quod esset bona: et divisit lucem a tenebris. 



In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search
import torch

try:
    assert torch.cuda.is_available()
    device = torch.device("cuda")
except:
    device = torch.device("cpu")
print("Using device:", device)

model_checkpoint = f"{folder}/models/EEPS_emanjavacas-MacBERTh_2025-05-05/checkpoint-1560"
bi_encoder = SentenceTransformer(model_checkpoint)
model_name = "EEPS_emanjavacas-MacBERTh_2025-05-05_checkpoint-1560"
bible_vectors = torch.load(f'{folder}/EEPS/Bibles_{model_name}.pt',map_location=device)

In [36]:
to_remove = ['1 Kings 10.18 (Douay-Rheims) - 1', '1 Kings 7.18 (AKJV) - 1', '2 Kings 5.8 (Douay-Rheims) - 1',
             '2 Paralipomenon 34.26 (Douay-Rheims) - 1', '4 Kings 19.20 (Douay-Rheims) - 1', '4 Kings 22.18 (Douay-Rheims) - 1',
             'Acts 2.16 (ODRV)', 'Ezekiel 23.1 (AKJV)', 'Ezekiel 28.1 (AKJV)', 'Ezekiel 28.1 (Geneva)', 'Ezekiel 30.1 (AKJV)',
             'Ezekiel 33.23 (AKJV)', 'Ezekiel 37.15 (Geneva)', 'Isaiah 37.21 (Douay-Rheims) - 1', 'Jeremiah 1.4 (AKJV)',
             'Jeremiah 11.1 (Douay-Rheims)', 'Jeremiah 11.1 (Geneva)', 'Jeremiah 11.3 (Douay-Rheims) - 1', 'Jeremiah 13.8 (Geneva)',
             'Jeremiah 16.9 (Douay-Rheims) - 0', 'Jeremiah 16.9 (Vulgate) - 0', 'Jeremiah 19.15 (Douay-Rheims) - 0',
             'Jeremiah 19.3 (AKJV) - 1', 'Jeremiah 24.5 (Douay-Rheims) - 0', 'Jeremiah 25.15 (Douay-Rheims) - 0',
             'Jeremiah 25.27 (Douay-Rheims) - 1', 'Jeremiah 27.4 (Douay-Rheims) - 1', 'Jeremiah 28.14 (Douay-Rheims) - 0',
             'Jeremiah 28.14 (Vulgate) - 0', 'Jeremiah 28.2 (Douay-Rheims) - 0', 'Jeremiah 29.25 (Douay-Rheims) - 0',
             'Jeremiah 29.30 (Douay-Rheims)', 'Jeremiah 29.30 (Geneva)', 'Jeremiah 30.1 (Geneva)', 'Jeremiah 30.1 (Vulgate)',
             'Jeremiah 31.23 (Douay-Rheims) - 0', 'Jeremiah 32.14 (AKJV) - 0', 'Jeremiah 32.14 (Douay-Rheims) - 0',
             'Jeremiah 32.15 (AKJV) - 0', 'Jeremiah 32.15 (Douay-Rheims) - 0', 'Jeremiah 32.15 (Vulgate) - 0',
             'Jeremiah 32.26 (AKJV)', 'Jeremiah 32.26 (Douay-Rheims)', 'Jeremiah 32.26 (Geneva)', 'Jeremiah 33.19 (AKJV)',
             'Jeremiah 33.19 (Douay-Rheims)', 'Jeremiah 33.19 (Geneva)', 'Jeremiah 33.23 (Douay-Rheims)', 'Jeremiah 34.12 (AKJV)',
             'Jeremiah 34.12 (Geneva)', 'Jeremiah 34.13 (Douay-Rheims) - 0', 'Jeremiah 35.12 (AKJV)', 'Jeremiah 35.12 (Douay-Rheims)',
             'Jeremiah 35.12 (Geneva)', 'Jeremiah 35.13 (Douay-Rheims) - 0', 'Jeremiah 35.17 (Douay-Rheims) - 0',
             'Jeremiah 35.17 (Vulgate) - 0', 'Jeremiah 35.18 (Douay-Rheims) - 1', 'Jeremiah 35.19 (Douay-Rheims) - 0',
             'Jeremiah 37.5 (Douay-Rheims)', 'Jeremiah 37.5 (Vulgate)', 'Jeremiah 37.6 (Douay-Rheims) - 0',
             'Jeremiah 39.16 (Douay-Rheims) - 0', 'Jeremiah 42.15 (Douay-Rheims) - 1', 'Jeremiah 42.18 (Douay-Rheims) - 0',
             'Jeremiah 42.18 (Vulgate) - 0', 'Jeremiah 43.10 (Douay-Rheims) - 1', 'Jeremiah 44.11 (Douay-Rheims) - 0',
             'Jeremiah 44.2 (AKJV) - 0', 'Jeremiah 44.2 (Douay-Rheims) - 0', 'Jeremiah 44.7 (Douay-Rheims) - 0',
             'Jeremiah 44.7 (Vulgate) - 0', 'Jeremiah 5.14 (Douay-Rheims) - 0', 'Jeremiah 50.18 (AKJV) - 0',
             'Jeremiah 50.18 (Douay-Rheims) - 0', 'Jeremiah 51.33 (AKJV) - 0', 'Jeremiah 51.33 (Douay-Rheims) - 0',
             'Jeremiah 51.33 (Vulgate) - 0', 'Jeremiah 7.1 (Douay-Rheims)', 'Jeremiah 7.21 (Douay-Rheims) - 0',
             'Jeremiah 7.3 (AKJV) - 0', 'Jeremiah 7.3 (Douay-Rheims) - 0', 'Jeremiah 9.15 (Douay-Rheims) - 0',
             'Jeremiah 9.15 (Vulgate) - 0', 'Jeremiah 9.17 (Douay-Rheims) - 0', 'Joshua 7.13 (Douay-Rheims) - 2',
             'Judges 6.8 (Douay-Rheims) - 1', 'Luke 20.42 (ODRV) - 0', 'Numbers 21.27 (Douay-Rheims) - 0',
             'Psalms 108.1 (Geneva) - 0', 'Psalms 122.1 (Geneva) - 0', 'Psalms 124.1 (Geneva) - 0', 'Psalms 126.1 (Geneva) - 0',
             'Psalms 131.1 (Geneva) - 0', 'Psalms 133.1 (Geneva) - 0', 'Psalms 145.1 (Geneva) - 0']

In [43]:
vectors = bi_encoder.encode([bible[v_id] for v_id in to_remove], batch_size=1024, convert_to_tensor=True, show_progress_bar=True)
hits = semantic_search(vectors,bible_vectors,query_chunk_size=1000,top_k=300)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
to_remove_dict = {k:None for k in to_remove}
for idx, hitlist in enumerate(tqdm.tqdm(hits)):
  v_id = to_remove[idx]
  text = bible[v_id]
  for rank, hit in enumerate(hitlist):
    v2_id = bible_ids[hit['corpus_id']]
    verse_text = bible[v2_id]
    score = round(hit['score'],3)
    if score >= 0.7:
      to_remove_dict[v2_id] = None
len(to_remove_dict)

  0%|          | 0/96 [00:00<?, ?it/s]

751

In [46]:
output = []
for v2_id in to_remove_dict:
  output.append({'verse_id': v2_id, 'verse_text': bible[v2_id]})
output = pd.DataFrame(output)
output.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/overly_vague.csv",index=False)
output

,verse_id,verse_text
0,1 Kings 10.18 (Douay-Rheims) - 1,Part 2 of 1 Kings 10.18: Thus saith the Lord t...
1,1 Kings 7.18 (AKJV) - 1,Part 2 of 1 Kings 7.18: and so did he for the ...
2,2 Kings 5.8 (Douay-Rheims) - 1,Part 2 of 2 Kings 5.8: therefore it is said in...
3,2 Paralipomenon 34.26 (Douay-Rheims) - 1,Part 2 of 2 Paralipomenon 34.26: Thus saith th...
4,4 Kings 19.20 (Douay-Rheims) - 1,Part 2 of 4 Kings 19.20: Thus saith the Lord t...
...,...,...
746,Psalms 38.1 (ODRV),"Psalms 38.1 Vnto the end, to Idithun him selfe..."
747,Psalms 71.1 (Vulgate),"Psalms 71.1 Psalmus, in Salomonem."
748,Psalms 6.1 (ODRV),"Psalms 6.1 Vnto the end in songs, the Psalme o..."
749,Psalms 30.1 (Geneva) - 0,Part 1 of Psalms 30.1: A Psalme or song of the...


# Organize QP Datasets

In [ ]:
os.remove(f"{folder}/QP/GOLDEN/cross_refs_confident_check_SameVerse.csv")
sorted(os.listdir(f"{folder}/QP"))

In [ ]:
missing = pd.read_csv(f"{folder}/QP/cross_refs_check.csv").to_dict(orient="records")
books = ['Psalms', 'Esther', 'Daniel', 'Jeremiah','Ecclesiasticus','Judith']
seen = pd.read_csv(f"{folder}/QP/cross_refs_missing_CommonDiff.csv").to_dict(orient='records')
seen.extend(pd.read_csv(f"{folder}/QP/cross_refs_missing_Lowest.csv").to_dict(orient='records'))
seen = {(x['verse_id'],x['label']):None for x in seen}
missing.extend(pd.read_csv(f"{folder}/QP/cross_refs_missing.csv").to_dict(orient='records'))
len(seen)

22097

In [ ]:
all_pred_files = {fp:None for fp in os.listdir(f"{folder}/QP/predictions")}
all_pred_files

{'GOLDEN_SET_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 '.ipynb_checkpoints': None,
 'cross_refs_missing_Lowest_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'cross_refs_missing_CommonDiff_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'M_QP_HighestShort_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'M_QP_LowestLong_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'preE_qp_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'GOLDEN_SET_additions_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'M_QP_PossibleC_1_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'M_QP_PossibleC_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'preE_additions_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'M_QP_PossibleC_0.5_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'cross_refs_missing_CHECK_Qwen_unsloth-Qwen2.5-7B_P1.json': None,
 'GOLDEN_SET_DSV3_P0.json': None,
 'GOLDEN_SET_additions_DSV3_P0.json': None,
 'preE_additions_DSV3_P0.json': None,
 'cross_refs_missing_CommonDiff_DSV3_P0.json': None,
 'cross_refs_missing_CHECK_DSV3_P0.json': None,
 'cross_refs_missing_Lowest_D

In [ ]:
output = []

for x in tqdm(missing):
  k = x['verse_id']
  v = x['label']
  if (k,v) in seen or (v,k) in seen: continue
  if re.sub(r" \d+.\d+","",k.split(" (")[0]) != re.sub(r" \d+.\d+","",v.split(" (")[0]): continue # same book
  if not re.search("|".join(books),k) and not re.search("|".join(books),v): continue # in the books w/ commonly different numberings

  # if k.split(" (")[0] != v.split(" (")[0]: continue # same verse no.
  # if " - " in k.split("; ")[0] and " - " in v.split("; ")[0]:
  #   if k.split("; ")[0].split(" - ")[1] == v.split("; ")[0].split(" - ")[1]: continue # not diff parts from the same verse

  if x['score'] < 0.97 and not re.search(r"Vulgate|Wycliffe|Tyndale",k): continue
  if abs(len(x['text'])-len(x['verse_text'])) >= ((len(x['text'])+len(x['verse_text'])) // 2):
    continue # no large disparity in length

  output.append(x)
output = pd.DataFrame(output)
# output.to_csv(f"{folder}/QP/GOLDEN/cross_refs_confident_missing_SameVerse.csv",index=False)
output.to_csv(f"{folder}/QP/cross_refs_missing_CHECK.csv",index=False)

output

In [ ]:
remove = {k:None for k in remove}
inputs = ['GOLDEN_SET_additions','GOLDEN_SET',
          'preE_qp','preE_additions',
          'cross_refs_missing_Lowest','cross_refs_missing_CommonDiff','cross_refs_missing_CHECK',
          'M_QP_LowestLong','M_QP_HighestShort','M_QP_PossibleC'
          ]
inputs = ['parallel_predictions_SAMPLE_DSV3']
def get_preds(fp):
  preds = {}
  ds_fp = f"{fp}_DSV3.json"
  if 'parallel' not in fp:
    ds_fp = f"{fp}_DSV3_P0.json"
    with open(f"{folder}/QP/predictions/{fp}_Qwen_unsloth-Qwen2.5-7B_P1.json",'r') as f:
      preds = json.load(f)
  DS_preds = {}
  if ds_fp in all_pred_files:
    with open(f"{folder}/QP/predictions/{fp}_DSV3_P0.json",'r') as f:
      DS_preds.update(json.load(f))

  if "PossibleC" in fp:
    with open(f"{folder}/QP/predictions/{fp}_0.5_Qwen_unsloth-Qwen2.5-7B_P1.json",'r') as f:
      more_preds = json.load(f)
      more_preds = {str(int(idx)+len(preds)):v for idx, v in more_preds.items()}
      preds.update(more_preds)
    with open(f"{folder}/QP/predictions/{fp}_1_Qwen_unsloth-Qwen2.5-7B_P1.json",'r') as f:
      more_preds = json.load(f)
      more_preds = {str(int(idx)+len(preds)):v for idx, v in more_preds.items()}
      preds.update(more_preds)
  return preds, DS_preds

for fp in inputs:
  output = []
  def_negative_output = []
  if f"{fp}.csv" in os.listdir(f"{folder}/QP"):
    data = pd.read_csv(f"{folder}/QP/{fp}.csv").to_dict(orient='records')
  if "preE_qp" == fp:
    data.extend(pd.read_csv(f"{folder}/QP/preE_Vul.csv").to_dict(orient='records'))

  preds, DS_preds = get_preds(fp)

  for idx, entry in enumerate(data):
    v_id = str(entry['verse_id']).split("; ")[0]
    if v_id in remove:
      if " - " in v_id:
        new_key = v_id.split(" - ")[0]
        if new_key in bible_info:
          entry['verse_id'] = new_key
          entry['verse_text'] = bible_info[new_key][3]

    entry['prediction'] = preds[str(idx)].capitalize()
    if 'label' in entry:
      if fp != "GOLDEN_SET" and fp != "preE_qp":
        del entry['label']
    if len(DS_preds) == 0:
      if 'label' in entry:
        if str(entry['label']) == entry['prediction']:
          entry['match'] = True
        else:
          entry['match'] = False
    else:
      if str(idx) in DS_preds:
        entry['DS_prediction'] = DS_preds[str(idx)].capitalize()
        if entry['DS_prediction'] == entry['prediction']:
          entry['match'] = True
        else:
          entry['match'] = False
      else:
        entry['DS_prediction'] = ""


    entry['correction'] = ''
    if 'PossibleC' in fp:
      if entry['prediction'] == "True":
        entry['correction'] = False
    entry['verse_text'] = normalize_stylized(entry['verse_text'])
    entry['text'] = normalize_stylized(entry['text'])
    entry['length_disparity'] = False
    if abs(len(entry['text'])-len(entry['verse_text'])) >= ((len(entry['text'])+len(entry['verse_text'])) // 2):
      entry['length_disparity'] = True
    output.append(entry)
  output = pd.DataFrame(output)

  output = output.sort_values(by='text', key=lambda x: x.str.len(), ascending=False)
  output.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/{fp}.csv",index=False)
  print(fp, len(output))

In [ ]:
inputs = ['parallel_predictions_SAMPLE']
def get_preds(fp):
  preds = {}
  if f"{fp}_DSV3.json" in all_pred_files:
    with open(f"{folder}/QP/predictions/{fp}_DSV3.json",'r') as f:
      preds.update(json.load(f))
  return preds

for fp in inputs:
  output = []

  if f"{fp}.csv" in os.listdir(f"{folder}/QP"):
    data = pd.read_csv(f"{folder}/QP/{fp}.csv").to_dict(orient='records')
  preds = get_preds(fp)

  for idx, entry in enumerate(data):
    entry['prediction'] = ''
    if str(idx) in preds:
      entry['prediction'] = preds[str(idx)].capitalize()
    output.append(entry)

  output = pd.DataFrame(output)
  output.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/{fp}.csv",index=False)
  print(fp, len(output))

parallel_predictions_SAMPLE 14297


In [ ]:
output

,index,version,text,verse_text,score,label,prediction
0,0,ODRV,Genesis 2.8 And the LORD God planted a garden ...,Genesis 2.8 And our Lord God had planted a Par...,0.970,NaN,True
1,0,Vulgate,Genesis 2.8 And the LORD God planted a garden ...,Genesis 2.8 Plantaverat autem Dominus Deus par...,0.934,NaN,True
2,0,Wycliffe,Genesis 2.8 And the LORD God planted a garden ...,Genesis 2.8 Forsothe the Lord God plauntide at...,0.922,NaN,True
3,0,Geneva,Genesis 2.8 And the LORD God planted a garden ...,Genesis 2.8 And the Lord God planted a garden ...,0.996,NaN,True
4,0,AKJV,Genesis 2.8 And the LORD God planted a garden ...,Genesis 2.8 And the LORD God planted a garden ...,0.995,NaN,True
...,...,...,...,...,...,...,...
14292,2499,ODRV,Revelation 21.4 He will wipe away every tear f...,Revelation 21.4 And God shal wipe away al tear...,0.950,NaN,True
14293,2499,Tyndale,Revelation 21.4 He will wipe away every tear f...,Revelation 21.4 And God shall wype awaye all t...,0.925,NaN,True
14294,2499,Vulgate,Revelation 21.4 He will wipe away every tear f...,Revelation 21.4 et absterget Deus omnem lacrim...,0.879,NaN,True
14295,2499,Geneva,Revelation 21.4 He will wipe away every tear f...,Revelation 21.4 And God shall wipe away all te...,0.960,NaN,True


In [ ]:
f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/{fp}.csv"confident = pd.read_csv(f"{folder}/QP/GOLDEN/cross_refs_confident.csv").to_dict(orient='records')
confident.extend(pd.read_csv(f"{folder}/QP/GOLDEN/cross_refs_confident_missing_SameVerse.csv").to_dict(orient='records'))
confident_dict = {}
for entry in confident:
  v = entry['verse_id']
  k = entry['label']
  if k not in confident_dict:
    confident_dict[k] = {}
  if v not in confident_dict:
    confident_dict[v] = {}
  confident_dict[k][v] = None
  confident_dict[v][k] = None

In [ ]:
item = "Jeremiah 40.4"
for v_id in confident_dict:
  if item in v_id:
    print(confident_dict[v_id])

{'Jeremiah 40.4 (Geneva)': None}
{'Jeremiah 40.4 (AKJV)': None}
{'Jeremiah 40.4 (Geneva) - 3': None}
{'Jeremiah 40.4 (AKJV) - 4': None}


In [ ]:
bible_info['Jeremiah 40.4 (Geneva) - 3']

('Geneva',
 'Old Testament',
 'Jeremiah',
 'whither it seemeth good, and conuenient for thee to goe, thither goe.')

# Edit mined data for annotations

In [ ]:
# input = "GOLDEN_SET"
# input = "preE"
input = "M_seg"

if "GOLD" in input:
  GOLD = pd.read_csv(f"{folder}/QP/{input}.csv").to_dict(orient="records")
elif "preE" == input:
  GOLD = pd.read_csv(f"{folder}/QP/preE_qp.csv").to_dict(orient="records")
  GOLD.extend(pd.read_csv(f"{folder}/QP/preE_Vul.csv").to_dict(orient="records"))
additions = pd.read_csv(f"{folder}/EEPS/qp/{input}_additional_0.csv").to_dict(orient="records")
len(GOLD), len(additions)

(5157, 41119)

In [ ]:
corpus = {}
for idx, entry in enumerate(GOLD):
  key = entry['text'].strip()
  if key not in corpus:
    corpus[key] = {"positives":[],"negatives":[],"preds":[]}
  if entry['label'] is True:
    if not isinstance(entry['verse_id'],float):
      for v_id in entry['verse_id'].split("; "):
        corpus[key]['positives'].append(v_id)
    else:
      corpus[key]['positives'].append("No Bible verse quotation or paraphrase found")
  elif entry['label'] is False:
    for v_id in entry['verse_id'].split("; "):
      corpus[key]['negatives'].append(v_id)

for entry in additions:
  key = entry['text'].strip()

  if len(corpus[key]["preds"]) == 0:
      corpus[key]["preds"] = [entry]

  if input == "preE":
    if len(corpus[key]["preds"]) > 0:
      prev_score = corpus[key]["preds"][0]["label"]
      if entry["label"] > prev_score:
        corpus[key]["preds"] = [entry]
  else:
    v_ids = [x['verse_id'] for x in corpus[key]['preds']]
    scores = [x['label'] for x in corpus[key]['preds']]
    skip = False
    for idx, v_id in enumerate(v_ids):
      if entry['verse_id'].split(" - ")[0] in v_id:
        if scores[idx] < entry['label']:
          corpus[key]['preds'][idx] = entry
        skip = True
    if skip: continue
    entry['label'] = False
    corpus[key]['preds'].append(entry)
sum([len(x['preds']) for x in corpus.values()])

4591

In [ ]:
output = []
non_QP = "No Bible verse quotation or paraphrase found"
for key, x in corpus.items():
  if non_QP not in x["positives"]:
    if len(x['preds']) == 0:
      output.append({"verse_id":None, 'verse_text':non_QP,'text':key,'label':True})
      print(key)
  for entry in x['preds']:
    v_ids = entry['verse_id'].split("; ")
    skip = False
    for v_id in v_ids:
      v_id = entry['verse_id'].split(" - ")[0]
      if "GOLD" in input:
        if "No Bible verse quotation or paraphrase found" in x["positives"]:
          entry['label'] = False
          break
      for gold_v_id in x['positives']:
        if v_id in gold_v_id:
          skip = True
      for gold_v_id in x['negatives']:
        if v_id in gold_v_id:
          skip = True
    if skip: continue
    output.append(entry)
output = pd.DataFrame(output)
output = output.drop(columns=[x for x in output.columns.values if x not in ['verse_id','verse_text','text','label']])
output.to_csv(f"{folder}/EEPS/qp/{input}_edited.csv",index=False)

In [ ]:
print(input)
output

GOLDEN_SET


,verse_id,text,verse_text,label
0,1 Maccabees 12.11 (AKJV),"Our callings, our labours, our actions, and th...","We therefore at all times without ceasing, bot...",False
1,2 Chronicles 20.9 (AKJV),"Our callings, our labours, our actions, and th...","If, when euill commeth vpon vs, as the sword, ...",False
2,Judith 8.24 (AKJV),"Our callings, our labours, our actions, and th...","Now therefore, O brethren, let vs shew an exam...",False
3,Ecclesiasticus 1.1 (AKJV) - 12,"Our callings, our labours, our actions, and th...","and not onely these things, but the Law it sel...",False
4,1 Maccabees 12.15 (AKJV),"Our callings, our labours, our actions, and th...",For wee haue helpe from heauen that succoureth...,False
...,...,...,...,...
1039,1 Peter 4.3 (ODRV),Manifesta sunt opera carnis &c. The dedes of t...,For the time past sufficeth (to accomplish the...,False
1040,Matthew 15.19 (ODRV),Manifesta sunt opera carnis &c. The dedes of t...,"For from the hart come forth euil cogitations,...",False
1041,Colossians 3.5 (ODRV),Manifesta sunt opera carnis &c. The dedes of t...,Mortifie therfore your members that are vpon t...,False
1042,Wisdom 14.12 (AKJV),Manifesta sunt opera carnis &c. The dedes of t...,For the deuising of idoles was the beginning o...,False


In [ ]:
print(input)
output

preE


,verse_id,text,verse_text,label
0,Matthew 1.2 (Tyndale) - 1,Abraham begatte Isaac. Isaac begat Iacob.,Isaac begat Iacob:,0.954
1,1 Thessalonians 5.20 (Vulgate),et in prophetis meis nolite malignari.,Prophetias nolite spernere.,0.940
2,2 Timothy 2.20 (Tyndale) - 0,are not only vessels of gold and syluer,Not withstondinge in a greate housse are not o...,0.918
3,Genesis 21.25 (Wycliffe),This had straake Hierusale wt pestilencis.lxx.,"And he blamyde Abymelech for the pit of watir,...",0.929
4,Mark 14.54 (Wycliffe) - 0,T He mayster of setece in ye seconde boke.,But Petir suede hym afer in to the halle of th...,0.916
...,...,...,...,...
4580,1 John 5.12 (Vulgate) - 1,Non porest habere deum patrem qui non ◊◊ ◊◊.,"qui non habet Filium, vitam non habet.",0.948
4581,1 Samuel 26.3 (Vulgate) - 2,"Videns autem ◊ turbas ascendit in montem,",Videns autem quod venisset Saul post se in des...,0.964
4582,Genesis 37.5 (Vulgate) - 1,Causa cu ◊ turba secuta fuit).,quæ causa majoris odii seminarium fuit.,0.948
4583,Luke 20.42 (ODRV) - 0,S. ◊ stin vpon the. 99. Psalme.,and Dauid himseld saith in the booke of Psalmes:,0.905


# Organize mined data for segments

In [ ]:
input = "M_QP"

preds = []
for fp in os.listdir(f"{folder}/EEPS/qp"):
  if input in fp:
    preds.extend(pd.read_csv(f"{folder}/EEPS/qp/{fp}").to_dict(orient="records"))

df = pd.DataFrame(preds)
df = df.drop(columns=['Unnamed: 0'])
remove = {k:None for k in remove}
df = df[~df['verse_id'].isin(remove)]
print(len(df))

209005


In [ ]:
import sys
sys.path.append(f'Early-Modern-Sermons')
from abbreviations import *
from EEPS_citationHelper import *
abbrev_dict = {}
for a,blist in abbrev.items():
  if a in ["the","de",'ch']: continue
  abbrev_dict[a.capitalize()] = None
  for b in blist:
    if b in ["the","de",'ch']: continue
    abbrev_dict[b.capitalize()] = None
print(len(abbrev_dict))

1718


In [ ]:
folder = "./content/drive/MyDrive/DH/QP"

In [ ]:
import string

filtered = df[
    df['text']
    .str.split()
    .str[0]
    .str.strip(string.punctuation)
    .str.capitalize()
    .isin(abbrev_dict)
]

filtered.to_csv(f"{folder}/QP/{input}_PossibleC.csv")
filtered

,verse_id,score,text,verse_text
2,Ephesians 4.8 (Vulgate) - 1,0.922003,Ephesians iiij Captiuā duxit captiuitatem.,"Ascendens in altum, captivam duxit captivitatem:"
3,Psalms 149.9 (Vulgate),0.932108,Psalm. lxxxiiij. Misericordia et veritas obui ...,ut faciant in eis judicium conscriptum: gloria...
11,Nehemiah 12.5 (Vulgate); Nehemiah 12.5 (Douay-...,0.888296,"Psa, 14, Exceed, 23","Miamin, Madia, Belga,"
18,Psalms 48.1 (Vulgate); Psalms 84.1 (Vulgate),0.882790,"psa, 13, &, 52,","In finem, filiis Core. Psalmus."
24,Psalms 51.1 (Vulgate),0.943680,"Job, 〈 ◊ 〉,","In finem. Intellectus David,"
...,...,...,...,...
249040,Acts 25.16 (Vulgate),0.952919,"Ex hoc loco Manifest, eosdem dici & suisse Epi...",Ad quos respondi: Quia non est Romanis consuet...
249043,1 Chronicles 24.11 (Douay-Rheims),0.869988,"Deuteronomy 6. 4. Mark. 12. 29. 30, 31.","The ninth to Jesua, the tenth to Sechenia,"
249046,John 1.29 (Wycliffe) - 1,0.884971,"Rom. 15 ▪ 6, 5.",the lomb of God;
249050,Numbers 21.17 (Douay-Rheims) - 1,0.906277,Psalm 34 ▪ 3.,Let the well spring up.


In [ ]:
filtered = df[
    ~df['text']
    .str.split()
    .str[0]
    .str.strip(string.punctuation)
    .str.capitalize()
    .isin(abbrev_dict)
]
filtered = filtered[(filtered['score'] >= 0.92)]
filtered = filtered[filtered['text'].str.split().str.len() < 5]
filtered.to_csv(f"{folder}/QP/{input}_HighestShort.csv")
filtered

,verse_id,score,text,verse_text
21,Numbers 21.18 (Vulgate) - 1,0.924712,"〈 ◊ 〉, 24.","De solitudine, Matthana."
58,Proverbs 14.13 (Vulgate),0.959387,Extrema Gaudii Luctus occupat.,"Risus dolore miscebitur, et extrema gaudii luc..."
65,Exodus 29.20 (Wycliffe) - 1,0.947708,"The veapens, of faith",and of her riyt foot;
74,Ecclesiastes 10.4 (Douay-Rheims) - 1,0.921345,Fearfulness does most hurt,because care will make the greatest sins to ce...
99,2 Kings 19.22 (Vulgate) - 2,0.922835,Against Adamantius circa 12,Contra Sanctum Israël.
...,...,...,...,...
249029,Psalms 38.1 (Vulgate) - 0,0.924003,A. N. p. 52.,"In finem, ipsi Idithun."
249030,Psalms 38.1 (Vulgate) - 0,0.924524,A. N. p. 56.,"In finem, ipsi Idithun."
249031,Psalms 61.1 (Vulgate) - 0; Psalms 76.1 (Vulgat...,0.920520,A. N. p. 82.,"In finem, pro Idithun."
249033,Psalms 61.1 (Vulgate) - 0; Psalms 76.1 (Vulgat...,0.923333,A. N. p. 83.,"In finem, pro Idithun."


In [ ]:
filtered = df[
    ~df['text']
    .str.split()
    .str[0]
    .str.strip(string.punctuation)
    .str.capitalize()
    .isin(abbrev_dict)
]
filtered = filtered[(filtered['score'] < 0.92)]
filtered = filtered[filtered['text'].str.split().str.len() >= 10]
filtered.to_csv(f"{folder}/QP/{input}_LowestLong.csv")
filtered

,verse_id,score,text,verse_text
57,Psalms 87.1 (Vulgate),0.902873,"Sapie. iii. ‡ John viii, a Psalm i. b Pro x. a...","Canticum Psalmi, filiis Core, in finem, pro Ma..."
90,Ecclesiasticus 15.12 (Douay-Rheims) - 2,0.919258,God can not abide that we make much of creatures,for he hath no need of wicked men.
98,Luke 2.2 (ODRV),0.918624,In the see second book against the denatists c...,This first enroling was made by the President ...
164,Luke 20.42 (ODRV) - 0,0.912805,Saint Ambrose also in the 5. book of his Hexam...,and Dauid himseld saith in the booke of Psalmes:
210,Luke 22.20 (AKJV),0.917506,The sacrament is called (and Containeth in ver...,"Likewise also the cup after supper, saying, Th..."
...,...,...,...,...
248943,Leviticus 25.11 (Douay-Rheims) - 0,0.909893,See my 2d part on Saint Iohns First Epistle th...,Because it is the jubilee and the fiftieth year.
248955,Revelation 15.1 (ODRV) - 2,0.914682,The Elect were once under the Curse of the Law.,Because in them the wrath of God is consummate.
248967,John 11.52 (ODRV),0.916196,There was a federal Ʋnion of the Elect with ch...,"And not only for tht Nation, but to gather int..."
248979,2 Maccabees 4.11 (Douay-Rheims),0.919101,Rivet alleges 30 of the Reformed Divines beyon...,"And abolishing those things, which had been de..."


 # Edit mined cross references

In [ ]:
with open(f"{folder}/EEPS/qp/cross_refs_MacBERTh.json","r") as file:
  cross_refs = json.load(file)

In [ ]:
def get_missing_verses(BCV, versions):
  output = []
  for item in BCV:
    for b in b_versions:
      if b not in versions[item]:
        if f"{item} ({b})" in bible_info:
          output.append(bible_info[f"{item} ({b})"][3])
  return list(set(output))

In [ ]:
confident_CR = {}
toCheck_CR = {}
missing_CR = {}
from tqdm import tqdm
for v1, CR in tqdm(cross_refs.items()):
  v1 = re.sub("\s+"," ",v1)
  v1 = normalize_stylized(v1)
  if v1 not in bible: continue
  v1_ids = bible[v1]
  BCV = [v.split(" (")[0] for v in v1_ids]
  versions = {}
  for idx, b in enumerate(BCV):
    if b not in versions:
      versions[b] = []
    versions[b].append(re.findall(r"\((.*?)\)",v1_ids[idx])[0])

  v1_ids = "; ".join(v1_ids)
  max_item = (float('-inf'),None)

  for v in CR:
    score = CR[v]
    v = re.sub("\s+"," ",v)
    v = normalize_stylized(v)
    if v not in bible: continue
    v_ids = bible[v]

    v_ids = "; ".join(v_ids)
    if v_ids == v1_ids:
      continue
    skip = False
    if " - " in v1_ids and " - " not in v_ids: continue
    if " - " not in v1_ids and " - " in v_ids: continue

    for item in BCV:
      if item in v_ids:
        item_ver = re.findall(rf"{item} \((.*?)\)",v_ids)
        for i in item_ver:
          versions[item].append(i)
        skip = True
    if skip and score >= 0.98:
      if (v1,v) not in confident_CR:
        confident_CR[(v1,v)] = None
      continue
    if score > max_item[0]:
      max_item = (score,(v1,v))

  if max_item[1] is not None:
    if (v1,v) not in toCheck_CR:
        toCheck_CR[(v1,v)] = max_item[0]

  missing = get_missing_verses(BCV, versions)
  for m in missing:
    if (v1,m) not in missing_CR:
        missing_CR[(v1,m)] = None

100%|██████████| 327969/327969 [01:13<00:00, 4455.88it/s]


In [ ]:
output = []
for k,v in confident_CR:
   k_ids = "; ".join(bible[k])
   v_ids = "; ".join(bible[v])
   output.append({'verse_id':v_ids,'text':k, 'verse_text':v, 'label':k_ids})
output = pd.DataFrame(output)
output.to_csv(f"{folder}/QP/GOLDEN/cross_refs_confident.csv",index=False)
output

,verse_id,text,verse_text,label
0,Genesis 1.1 (Geneva),"In the beginning God created the Heauen, and t...",In the beginning God created the heauen and th...,Genesis 1.1 (AKJV)
1,Genesis 1.1 (ODRV),"In the beginning God created the Heauen, and t...",IN THE beginning GOD created heauen and earth.,Genesis 1.1 (AKJV)
2,Genesis 1.2 (Geneva),"And the earth was without forme, and voyd, and...","And the earth was without forme and void, and ...",Genesis 1.2 (AKJV)
3,Genesis 1.3 (Geneva),"And God said, Let there be light: and there wa...","Then God said, Let there be light: And there w...",Genesis 1.3 (AKJV)
4,Genesis 1.3 (Geneva) - 0,"And God said, Let there be light:","Then God said, Let there be light:",Genesis 1.3 (AKJV) - 0
...,...,...,...,...
90126,Matthew 8.9 (AKJV) - 2; Matthew 8.9 (Geneva) - 2,"and to another, Come, and he cometh;","and to another, Come, and he commeth:",Matthew 8.9 (Wycliffe) - 2
90127,Matthew 18.18 (Wycliffe) - 1,"Y seie to you treuli, what euer thingis ye byn...","and what euer thingis ye vnbynden on erthe, th...",Matthew 18.18 (Wycliffe) - 0
90128,Matthew 18.18 (Wycliffe) - 0,"and what euer thingis ye vnbynden on erthe, th...","Y seie to you treuli, what euer thingis ye byn...",Matthew 18.18 (Wycliffe) - 1
90129,Matthew 26.58 (Tyndale) - 1,"and he wente in, and sat with the seruauntis, ...",and went in and sate with the servauntes to se...,Matthew 26.58 (Wycliffe) - 1


In [ ]:
toCheck_df = []
for v,k in toCheck_CR:
   score = toCheck_CR[(v,k)]
   if k not in bible: continue
   if v not in bible: continue
   k_ids = "; ".join(bible[k])
   v_ids = "; ".join(bible[v])
   if " - " in k_ids and " - " not in v_ids: continue
   if " - " not in k_ids and " - " in v_ids: continue
   toCheck_df.append({'verse_id':k_ids,'text':v, 'verse_text':k, 'label':v_ids, 'score':score})
toCheck_df = pd.DataFrame(toCheck_df)
toCheck_df.to_csv(f"{folder}/QP/cross_refs_check.csv",index=False)
toCheck_df

,verse_id,text,verse_text,label,score
0,Genesis 1.7 (Geneva),"And the earth was without forme, and voyd, and...","Then God made the firmament, and separated the...",Genesis 1.2 (AKJV),0.969885
1,Genesis 7.18 (AKJV) - 1,and the Spirit of God mooued vpon the face of ...,and the Arke went vpon the face of the waters.,Genesis 1.2 (AKJV) - 1,0.962760
2,Genesis 1.6 (AKJV) - 0,"And God said, Let there be light:","And God said, Let there be a firmament in the ...",Genesis 1.3 (AKJV) - 0,0.937474
3,Genesis 1.7 (Geneva),"And God said, Let there be a firmament in the ...","Then God made the firmament, and separated the...",Genesis 1.6 (AKJV),0.970065
4,Psalms 78.16 (Geneva) - 1,and let it diuide the waters from the waters.,so that hee made the waters to descend like th...,Genesis 1.6 (AKJV) - 1,0.966029
...,...,...,...,...,...
194286,Genesis 31.52 (Wycliffe),"Jhesus seith to him, So I wole that he dwelle ...","sotheli this heep, and stoon be in to witnessy...",John 21.22 (Wycliffe),0.968061
194287,Mark 10.51 (Wycliffe) - 0,"Jhesus seith to him, So I wole that he dwelle ...","And Jhesus answeride, and seide to hym, What w...",John 21.22 (Wycliffe) - 0,0.974719
194288,Numbers 5.2 (Wycliffe) - 0,Therfor this word wente out among the brithere...,that thei caste out of the castels ech leprous...,John 21.23 (Wycliffe) - 0,0.947085
194289,Matthew 17.12 (Wycliffe),"This is thilke disciple, that berith witnessyn...","And Y seie to you, that Elie is nowe comun, an...",John 21.24 (Wycliffe),0.965312


In [ ]:
missing_df = []
for v,k in missing_CR:
   if k not in bible: continue
   if v not in bible: continue
   k_ids = "; ".join(bible[k])
   v_ids = "; ".join(bible[v])
   if " - " in k_ids and " - " not in v_ids: continue
   if " - " not in k_ids and " - " in v_ids: continue
   missing_df.append({'verse_id':k_ids,'text':v, 'verse_text':k, 'label':v_ids})
missing_df = pd.DataFrame(missing_df)
missing_df.to_csv(f"{folder}/QP/cross_refs_missing.csv",index=False)
missing_df

,verse_id,text,verse_text,label
0,Genesis 1.1 (Vulgate),"In the beginning God created the Heauen, and t...",In principio creavit Deus caelum et terram.,Genesis 1.1 (AKJV)
1,Genesis 1.1 (Wycliffe),"In the beginning God created the Heauen, and t...",In the bigynnyng God made of nouyt heuene and ...,Genesis 1.1 (AKJV)
2,Genesis 1.2 (Wycliffe),"And the earth was without forme, and voyd, and...","Forsothe the erthe was idel and voide, and der...",Genesis 1.2 (AKJV)
3,Genesis 1.2 (Vulgate),"And the earth was without forme, and voyd, and...","Terra autem erat inanis et vacua, et tenebrae ...",Genesis 1.2 (AKJV)
4,Genesis 1.3 (Vulgate),"And God said, Let there be light: and there wa...",Dixitque Deus: Fiat lux. Et facta est lux.,Genesis 1.3 (AKJV)
...,...,...,...,...
372245,John 21.25 (AKJV),And ther ben also manye othere thingis that Jh...,And there are also many other things which Ies...,John 21.25 (Wycliffe)
372246,John 21.25 (Vulgate),And ther ben also manye othere thingis that Jh...,Sunt autem et alia multa quae fecit Jesus: qua...,John 21.25 (Wycliffe)
372247,John 21.25 (Geneva),And ther ben also manye othere thingis that Jh...,Nowe there are also many other things which Ie...,John 21.25 (Wycliffe)
372248,John 21.25 (Tyndale),And ther ben also manye othere thingis that Jh...,There are also many other thinges which Iesus ...,John 21.25 (Wycliffe)


# Add AKJV and ORDV to Old QP Data

In [ ]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
inputfname = "preE_qp"

data = pd.read_csv(f"../../QP/OLD_{inputfname}.csv")
data = data.to_dict(orient="records")

new_data = []
for item in data:
    seen = {}
    if re.search(r"\'",item['verse_id']):
        verse_ids = re.findall("\'(.*?)\'",item['verse_id'])
    else:
        verse_ids = [item['verse_id'].strip()]
    for v in verse_ids:
        if "KJV" in v:
            v = re.sub("KJV","AKJV",v)
        elif re.search("Douay",v):
            new_v = re.sub("Douay-Rheims","ODRV",v)
            if new_v in bible_info:
                v = new_v
        if v in bible_info:
            text = bible_info[v][3]
            if text not in seen:
                seen[text] = []
            if v not in seen[text]:
                seen[text].append(v)
        elif "-" in v:
            orig_v = v
            idx = int(v.split(" - ")[-1])
            v_id = v.split(" - ")[0]
            if idx <= 1:
                v = v_id
            else:
                v = f"{v_id} - {idx-1}"
            if re.search("Douay",v):
                new_v = re.sub("Douay-Rheims","ODRV",v)
                if new_v in bible_info:
                    v = new_v
            if orig_v == "John 6.59 (Douay-Rheims) - 2":
                v = "John 6.58 (ODRV) - 2"
            if v in bible_info:
                text = bible_info[v][3]
                if text not in seen:
                    seen[text] = []
                if v not in seen[text]:
                    seen[text].append(v)
            else:
                print(v,orig_v)
    for text, v_id in seen.items():
        new_item = item.copy()
        new_item['verse_text'] = text
        new_item['verse_id'] = "; ".join(v_id)
        new_data.append(new_item)
new_data = pd.DataFrame(new_data)
new_data = new_data.drop(columns=['score']) #
# new_data = new_data.drop(columns=['Unnamed: 0']) #
# new_data = new_data.drop(columns=['tcpID','sidx','loc','part','group']) #
new_data = new_data.drop_duplicates()
new_data.to_csv(f"../../QP/{inputfname}.csv")
new_data

In [ ]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
# inputfname = "preE_qp"

data = pd.read_csv(f"../../QP/OLD_{inputfname}.csv")
data = data.to_dict(orient="records")

new_data = []
for item in data:
    seen = {}
    if re.search(r"\'",item['verse_id']):
        verse_ids = re.findall("\'(.*?)\'",item['verse_id'])
    else:
        verse_ids = [item['verse_id'].strip()]
    for v in verse_ids:
        if "KJV" in v:
            v = re.sub("KJV","AKJV",v)
        elif re.search("Douay",v):
            new_v = re.sub("Douay-Rheims","ODRV",v)
            if new_v in bible_info:
                v = new_v
        if v in bible_info:
            text = bible_info[v][3]
            if text not in seen:
                seen[text] = []
            if v not in seen[text]:
                seen[text].append(v)
        elif "-" in v:
            orig_v = v
            idx = int(v.split(" - ")[-1])
            v_id = v.split(" - ")[0]
            if idx <= 1:
                v = v_id
            else:
                v = f"{v_id} - {idx-1}"
            if re.search("Douay",v):
                new_v = re.sub("Douay-Rheims","ODRV",v)
                if new_v in bible_info:
                    v = new_v
            if v in bible_info:
                text = bible_info[v][3]
                if text not in seen:
                    seen[text] = []
                if v not in seen[text]:
                    seen[text].append(v)
            else:
                print(v,orig_v)
    for text, v_id in seen.items():
        new_item = item.copy()
        new_item['verse_text'] = text
        new_item['verse_id'] = "; ".join(v_id)
        new_data.append(new_item)
new_data = pd.DataFrame(new_data)
new_data = new_data.drop(columns=['score']) #
# new_data = new_data.drop(columns=['Unnamed: 0']) #
# new_data = new_data.drop(columns=['tcpID','sidx','loc','part','group']) #
new_data = new_data.drop_duplicates()
new_data.to_csv(f"../../QP/{inputfname}.csv")
new_data